# Installation of JanusQ
JanusQ can be installed by docker, wheel, and soruce code. Docker is recommended, as all functions have been tested on it.
Linux (Ubuntu 22.04 latest) platform and Python (3.10) is prefered when instaling by wheel or soruce code.

**From docker.**
Pull docker using docker 
        
        docker pull jxhhhh/janusq:latest

The code is in "/home/JanusQ-main". The examples that can be directly run is in "/home/JanusQ-main/examples"

**From wheel.**
Download janusq.whl from "JanusQ-main/dist".

        pip install janusq.whl

**From source code.**
Run following commends.

        git clone git@github.com:JanusQ/JanusQ.git
        cd JanusQ
        pip install -r requirements.txt

# Structure of JanusQ

- JanusCT
  - vectorization
  - fidelity preidiction
  - fidelity optimization
  - unitary decomposition
  - bug identification
- JanusFEM
  - readout calibration
- HyQSAT
  - solve sat domain problem
- time crystal

# Note
1. Permission denied 'minisat_core'  
chmod +x janusq/hyqsat/minisat_core

# Test JanusQ

In [1]:
import sys
sys.path.append('..')

#### Test Janus-CT

In [2]:
# test janus-ct
from janusq.data_objects.backend import LinearBackend
from janusq.data_objects.random_circuit import random_circuits
from janusq.analysis.vectorization import RandomwalkModel
# define a 10-qubit backend
n_qubits = 8
n_steps = 1
n_walks = 20
backend = LinearBackend(n_qubits, 1)

circuits = random_circuits(backend, n_circuits=300, n_gate_list=[30, 50, 100], two_qubit_prob_list=[.4], reverse=True)

vec_model = RandomwalkModel(n_steps = n_steps, n_walks = n_walks, backend = backend)
vec_model.train(circuits, multi_process=False, remove_redundancy = False)

2024-04-15 12:04:12,846 - INFO: start random walk for 300 circuits
100%|██████████| 300/300 [00:01<00:00, 151.75it/s]
2024-04-15 12:04:14,828 - INFO: count path
2024-04-15 12:04:14,859 - INFO: device size after random walk = 15
2024-04-15 12:04:14,859 - INFO: 0's path table size = 11
2024-04-15 12:04:14,860 - INFO: 1's path table size = 16
2024-04-15 12:04:14,861 - INFO: 2's path table size = 19
2024-04-15 12:04:14,862 - INFO: 3's path table size = 19
2024-04-15 12:04:14,862 - INFO: 4's path table size = 19
2024-04-15 12:04:14,863 - INFO: 5's path table size = 19
2024-04-15 12:04:14,863 - INFO: 6's path table size = 16
2024-04-15 12:04:14,864 - INFO: 7's path table size = 11
2024-04-15 12:04:14,864 - INFO: (0, 1)'s path table size = 13
2024-04-15 12:04:14,866 - INFO: (1, 2)'s path table size = 18
2024-04-15 12:04:14,866 - INFO: (3, 4)'s path table size = 21
2024-04-15 12:04:14,867 - INFO: (2, 3)'s path table size = 21
2024-04-15 12:04:14,867 - INFO: (6, 7)'s path table size = 13
2024-0

#### Test Janus-FEM

In [3]:
# test readout calibration

from janusq.simulator.noisy_simulator import NoisySimulator
from janusq.data_objects.backend import LinearBackend
from janusq.simulator.readout_error_model import ReadoutErrorModel
from janusq.optimizations.readout_mitigation.fem import  EnumeratedProtocol
from janusq.data_objects.algorithms import get_algs

n_qubits = 3
backend = LinearBackend(n_qubits, 1)
circuit = get_algs(n_qubits, backend, algs = ['ghz'])[0]

samples = 10000

simulator = NoisySimulator(
    backend, readout_error_model = ReadoutErrorModel.random_model(backend))
output_noise = simulator.execute(circuit,samples)


protocol = EnumeratedProtocol(n_qubits)
real_bstrs, circuits_protocol = protocol.gen_circuits()

all_statuscnts = [
    simulator.execute(cir_protocol, samples)
    for cir_protocol in circuits_protocol
]

print(all_statuscnts)


2024-04-15 12:04:14,992 - INFO: Pass: UnrollCustomDefinitions - 0.07510 (ms)
2024-04-15 12:04:14,993 - INFO: Begin BasisTranslator from source basis {('u3', 1)} to target basis {'u', 'ry', 'reset', 'z', 'delay', 'x', 'measure', 'barrier', 'p', 'rz', 'rx', 'cx', 'snapshot'}.
2024-04-15 12:04:14,994 - DEBUG: Begining basis search from {('u3', 1)} to {'u', 'ry', 'reset', 'z', 'delay', 'x', 'measure', 'barrier', 'p', 'rz', 'rx', 'cx', 'snapshot'}.


2024-04-15 12:04:14,994 - DEBUG: Gate u1 generated using rule 
global phase: theta/2
   ┌───────────┐
q: ┤ Rz(theta) ├
   └───────────┘
 with total cost of 1.0.
2024-04-15 12:04:15,174 - DEBUG: Gate id generated using rule 
   ┌───────┐
q: ┤ Rz(0) ├
   └───────┘
 with total cost of 1.0.
2024-04-15 12:04:15,176 - DEBUG: Gate tdg generated using rule 
   ┌──────────┐
q: ┤ U1(-π/4) ├
   └──────────┘
 with total cost of 1.0.
2024-04-15 12:04:15,176 - DEBUG: Gate s generated using rule 
   ┌─────────┐
q: ┤ U1(π/2) ├
   └─────────┘
 with total cost of 1.0.
2024-04-15 12:04:15,177 - DEBUG: Gate u3 generated using rule 
   ┌──────────────────┐
q: ┤ U(theta,phi,lam) ├
   └──────────────────┘
 with total cost of 1.0.
2024-04-15 12:04:15,178 - DEBUG: Transformation path:
2024-04-15 12:04:15,179 - DEBUG: u3/1 => [Parameter(theta), Parameter(phi), Parameter(lam)]
   ┌──────────────────┐
q: ┤ U(theta,phi,lam) ├
   └──────────────────┘
2024-04-15 12:04:15,180 - DEBUG: s/1 => []
   ┌─────────┐
q: ┤ U1

[{'110': 27, '111': 4, '011': 28, '101': 50, '001': 597, '100': 715, '010': 362, '000': 8217}, {'110': 2, '100': 36, '111': 43, '011': 408, '101': 711, '010': 11, '000': 220, '001': 8569}, {'110': 37, '111': 1, '011': 34, '100': 696, '101': 65, '001': 678, '000': 8090, '010': 399}, {'001': 38, '101': 3, '100': 49, '111': 60, '011': 586, '110': 655, '000': 525, '010': 8084}, {'100': 1, '110': 26, '000': 18, '010': 245, '101': 40, '001': 553, '111': 738, '011': 8379}, {'101': 2, '001': 32, '100': 39, '110': 664, '011': 611, '111': 68, '000': 526, '010': 8058}, {'110': 29, '011': 29, '101': 54, '001': 681, '100': 709, '000': 8100, '010': 398}, {'110': 2, '100': 21, '111': 44, '011': 404, '101': 750, '010': 12, '000': 265, '001': 8502}, {'110': 31, '101': 56, '011': 33, '111': 3, '100': 648, '001': 640, '010': 371, '000': 8218}, {'001': 26, '011': 1, '111': 21, '110': 399, '101': 700, '010': 15, '000': 326, '100': 8512}, {'010': 1, '000': 11, '110': 12, '011': 14, '111': 392, '100': 258, '

#### Test Janus-SAT

In [4]:
# test hyqsat calibration
# input cnf flie
file_path = "cnf_examples/uf50-01.cnf"
from janusq.hyqsat import readCNF

print(readCNF(file_path))

([(-3, 7, 36), (-3, -42, -48), (-41, -47, -49), (8, 17, -40), (-21, -31, -39), (-22, 36, 49), (14, 27, 38), (6, 15, -18), (6, 7, -43), (-7, 23, 34), (2, -13, 14), (2, -42, 47), (3, -33, -35), (40, 44, 49), (31, 36, 50), (-3, -36, -37), (26, -29, 43), (15, 29, -45), (-11, 18, 24), (6, -26, -47), (-10, -33, -50), (6, 16, 32), (-34, 37, 41), (7, -17, -28), (19, -44, 46), (7, 22, -48), (3, 34, 39), (31, -43, 46), (23, -27, 32), (-18, 37, -50), (5, 11, 20), (6, -24, -45), (-14, -23, -34), (20, 21, -22), (-17, 24, 50), (-24, -25, -27), (3, 21, 35), (-26, -36, 47), (-28, -45, 49), (-6, 12, -21), (-15, -17, -39), (2, -14, 41), (-23, 25, 36), (-3, -39, -40), (20, 35, 50), (27, 31, -39), (-15, -40, 45), (34, 35, 50), (-1, 12, -48), (18, -30, -35), (-24, -25, 27), (-4, -12, -33), (-24, -37, -43), (31, -37, -44), (-9, 14, -38), (-16, 33, 34), (4, -5, -35), (-3, -19, -21), (-29, -35, -36), (7, 36, -43), (14, 30, 41), (-7, -24, -35), (6, 35, -42), (-1, -15, 39), (-16, 27, 49), (-10, -37, 49), (-3, -